Image steganography based on canny,sobal,prewitt edge detection technique.

Algorithm

In [13]:
import numpy as np
import cv2
import hashlib

def text_to_binary(text):
    binary_str = ''.join(format(ord(char), '08b') for char in text)
    return binary_str

def logistic_map(seed, length, N):
    seq = []
    x = seed
    for _ in range(length):
        x = (N * x * (1 - x)) % 1
        seq.append(x)
    return seq

def sine_map(seed, length):
    seq = []
    x = seed
    for _ in range(length):
        x = np.sin(np.pi * x)
        seq.append(int(x * length))
    return seq

def select_bit_position(x, y):
    if x % 2 == 1 and y % 2 == 1:
        return (x + y) % 2
    elif x % 2 == 1 and y % 2 == 0:
        return (x + y**2) % 2
    elif x % 2 == 0 and y % 2 == 1:
        return (x**2 + y) % 2
    else:
        return (x**2 + y**2) % 2

    # Function to convert binary to text
def binary_to_text(binary_str):
    text = ''.join([chr(int(binary_str[i:i+8], 2)) for i in range(0, len(binary_str), 8)])
    return text
def edge_detection(image1):
    canny_edges = cv2.Canny(image1, 100, 50)
    sobel_x = cv2.Sobel(image1, cv2.CV_64F, 1, 0, ksize=5)
    sobel_y = cv2.Sobel(image1, cv2.CV_64F, 0, 1, ksize=5)
    sobel_edges = np.sqrt(sobel_x**2 + sobel_y**2).astype(np.uint8)
    kernelx = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
    kernely = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
    perwitt_x = cv2.filter2D(image1, -1, kernelx)
    perwitt_y = cv2.filter2D(image1, -1, kernely)
    perwitt_edges = np.sqrt(perwitt_x**2 + perwitt_y**2).astype(np.uint8)
    intersection_edges = cv2.bitwise_and(canny_edges, cv2.bitwise_and(sobel_edges, perwitt_edges))
    return intersection_edges
def encrypt(secret_text, image_path, output_path):
    binary_secret = text_to_binary(secret_text)
    length = len(binary_secret)

    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    edge_image = edge_detection(image)
    N = np.sum(edge_image > 0)

    seed = 0.5  # Adjust the seed value as needed
    seq = logistic_map(seed, length * 8, N)
    sq = sine_map(seed, length * 8)

    if len(binary_secret) < length * 8:
        binary_secret += '0' * (length * 8 - len(binary_secret))

    encrypted_binary = ''
    for i in range(length * 8):
        position = select_bit_position(sq[i], seq[i])
        encrypted_binary += binary_secret[i] if position == 0 else str(1 - int(binary_secret[i]))

    k = 0
    for i in range(len(image)):
        for j in range(len(image[i])):
            if edge_image[i][j] > 0:
                image[i][j] = int(f"{image[i][j]:07b}"[:-1] + encrypted_binary[k], 2)
                k += 1
                if k >= len(encrypted_binary):
                    break
        if k >= len(encrypted_binary):
            break

    cv2.imwrite(output_path, image)
    key1 = hashlib.sha512(secret_text.encode()).digest()
    key2 = hashlib.sha512(secret_text[::-1].encode()).digest()

    return encrypted_binary, key1, key2, length, sq, seq

# Modify the decrypt function to accept edge_image as a parameter


def decrypt(encrypted_binary, length, sq, seq, image_path, edge_image):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    decrypted_binary = ''
    k = 0
    for i in range(len(image)):
        for j in range(len(image[i])):
            if edge_image[i][j] > 0:
                if k < len(encrypted_binary):
                    decrypted_binary += f"{image[i][j]:08b}"[-1]
                    k += 1

    decrypted_text = ''
    for i in range(length * 8):
        position = select_bit_position(sq[i], seq[i])
        if i < len(decrypted_binary):
            decrypted_bit = decrypted_binary[i]
            if position == 0:
                decrypted_text += decrypted_bit
            else:
                decrypted_text += '1' if decrypted_bit == '0' else '0'

    decrypted_text = binary_to_text(decrypted_text)

    return decrypted_text

def calculate_max_text_size(image1):
    # Read the image
    image = cv2.imread("D:\8th sem\Project\Image_steganography_-based-_on-_edge_-detection-_techniques\Input_images\size_128\monalisa128.bmp")

    # Check if the image is loaded successfully
    if image is not None:
        # Get the dimensions of the image (height and width)
        height, width, _ = image.shape

        # Calculate the maximum number of characters that can be embedded
        max_text_size = height
        return max_text_size
    else:
        return 0  # Return 0 if the image cannot be loaded

def check_condition(x,y):
    if x < y/8:

        # Example usage:
        output_path = "D:\8th sem\Project\Image_steganography_-based-_on-_edge_-detection-_techniques\Output_images\size_128\mona1.bmp"

        encrypted_binary, key1, key2, length, sq, seq = encrypt(text, "D:\8th sem\Project\Image_steganography_-based-_on-_edge_-detection-_techniques\Input_images\size_128\monalisa128.bmp", output_path)
        #print("Encrypted Text:", encrypted_binary)

        #edge_image = cv2.Canny(cv2.imread("lina256.bmp", cv2.IMREAD_GRAYSCALE), 50, 150)
        decrypted_text = decrypt(encrypted_binary, length, sq, seq, output_path, edge_image)
        print("Decrypted Text:", decrypted_text)

    else:
        print("Length of secret text is out reaching the limiting Kindly reduce size of text")
        print("Length of text should be less than "+str(y/8))


text = "This is an encrypted message done This is an encrypted place This is an encrypted message done done"
print("length of text "+str(len(text)))
image1 = cv2.imread("D:\8th sem\Project\Image_steganography_-based-_on-_edge_-detection-_techniques\Input_images\size_128\monalisa128.bmp", cv2.IMREAD_GRAYSCALE)
edge_image = edge_detection(image1)
num_edges_detected = np.count_nonzero(edge_image)
print("Number of edges detected:", num_edges_detected)
#print(calculate_max_text_size(image1));
check_condition(len(text),num_edges_detected)


length of text 99
Number of edges detected: 1197
Decrypted Text: This is an encrypted message done This is an encrypted place This is an encrypted message done done                                                   


Image_Size : 256 * 256

In [2]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim

# Load the original and compressed images
original_image = cv2.imread('D:\Final-Year-Project\Image_steganography_-based-_on-_edge_-detection-_techniques\Input_images\size_128\lina128.bmp')
compressed_image = cv2.imread('D:\Final-Year-Project\Image_steganography_-based-_on-_edge_-detection-_techniques\Output_images\size_128\mona.bmp')

# Define the width and height for resizing
width = 256  # specify your desired width
height = 256  # specify your desired height

# Resize the images
resized_original_image = cv2.resize(original_image, (width, height))
resized_compressed_image = cv2.resize(compressed_image, (width, height))

mse = np.mean((resized_original_image - resized_compressed_image) ** 2)
psnr = 20 * np.log10(255 / np.sqrt(mse))

# Calculate MSE
mse = np.mean((resized_original_image - resized_compressed_image) ** 2)

print(f'PSNR: {psnr} dB')
print(f'MSE: {mse}')
# Calculate SSIM with a smaller win_size value (e.g., 3x3)
ssim_index, _ = ssim(resized_original_image, resized_compressed_image, win_size=3, full=True)

print(f'SSIM: {ssim_index}')

PSNR: 27.92626692134332 dB
MSE: 104.82229614257812
SSIM: 0.496831366069978


Image_size : 128 * 128

In [15]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim

# Load the original and compressed images
original_image = cv2.imread('D:\8th sem\Project\Image_steganography_-based-_on-_edge_-detection-_techniques\Input_images\size_128\monalisa128.bmp').astype(np.uint8)
compressed_image = cv2.imread('D:\8th sem\Project\Image_steganography_-based-_on-_edge_-detection-_techniques\Output_images\size_128\mona1.bmp').astype(np.uint8)

# Calculate MSE
mse = np.mean((original_image - compressed_image) ** 2)

# Calculate PSNR
psnr = 20 * np.log10(127 / np.sqrt(mse))

# Calculate SSIM with a smaller win_size value (e.g., 3x3)
ssim_index, _ = ssim(original_image, compressed_image, win_size=3, full=True)

# Scale SSIM to the range [0, 1]
ssim_index = (ssim_index + 1) / 2

print(f'PSNR: {psnr} dB')
print(f'MSE: {mse}')
print(f'SSIM: {ssim_index}')

PSNR: 56.381238019349155 dB
MSE: 0.037109375
SSIM: 0.9999288565528208


Image_size : 512 * 512

In [ ]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim

# Load the original and compressed images
original_image = cv2.imread('house512.bmp').astype(np.uint8)
compressed_image = cv2.imread('house512_100.bmp').astype(np.uint8)

# Calculate MSE
mse = np.mean((original_image - compressed_image) ** 2)

# Calculate PSNR
psnr = 20 * np.log10(511 / np.sqrt(mse))

# Calculate SSIM with a smaller win_size value (e.g., 3x3)
ssim_index, _ = ssim(original_image, compressed_image, win_size=3, full=True)

# Scale SSIM to the range [0, 1]
ssim_index = (ssim_index + 1) / 2

print(f'PSNR: {psnr} dB')
print(f'MSE: {mse}')
print(f'SSIM: {ssim_index}')